<a href="https://colab.research.google.com/github/arpitawad04/League-Of-Legends-Pre-Match-Winner-Prediction-And-Recommendation-of-Champion-Using-Python/blob/main/MSE_Logistic_Regression_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#Read required csv dataset file
df=pd.read_csv('/content/LOL_transformed_dataset_extended_withadded_attributes.csv')

In [ ]:
#See dimension of dataframe
df.shape

(7425, 128)

In [ ]:
#Check for null values
df.isnull().sum()

Gameid                 0
Blue_Team_id           0
bluetop_playerid       0
bluetop_player_name    0
bluetop_champion       0
                      ..
red_ban2               0
red_ban3               0
red_ban4               0
red_ban5               0
Results                0
Length: 128, dtype: int64

In [ ]:
df.head()

,Gameid,Blue_Team_id,bluetop_playerid,bluetop_player_name,bluetop_champion,bluetop_Game_played,bluetop_champ_win_rate,bluetop_KDA,bluetop_dmg_to_cham,bluetop_goldearned,bluetop_monsterkillsownjungle,bluetop_monsterkillsenemyjungle,bluetop_goldat10,bluejng_playerid,bluejng_playername,bluejng_champion,bluejng_Game_played,bluejng_champ_win_rate,bluejng_KDA,bluejng_dmg_to_cham,bluejng_goldearned,bluejng_monsterkillsownjungle,bluejng_monsterkillsenemyjungle,bluejng_goldat10,bluemid_playerid,bluemid_playername,bluemid_champion,bluemid_Game_played,bluemid_champ_win_rate,bluemid_KDA,bluemid_dmg_to_cham,bluemid_goldearned,bluemid_monsterkillsownjungle,bluemid_monsterkillsenemyjungle,bluemid_goldat10,bluebot_playerid,bluebot_playername,bluebot_champion,bluebot_Game_played,bluebot_champ_win_rate,...,redmid_dmg_to_cham,redmid_goldearned,redmid_monsterkillsownjungle,redmid_monsterkillsenemyjungle,redmid_goldat10,redbot_playerid,redbot_playername,redbot_champion,redbot_Game_played,redbot_champ_win_rate,redbot_KDA,redbot_dmg_to_cham,redbot_goldearned,redbot_monsterkillsownjungle,redbot_monsterkillsenemyjungle,redbot_goldat10,redsup_playerid,redsup_playername,redsup_champion,redsup_Game_played,redsup_champ_win_rate,redsup_KDA,redsup_dmg_to_cham,redsup_goldearned,redsup_monsterkillsownjungle,redsup_monsterkillsenemyjungle,redsup_goldat10,Red_team_kills,Red_team_deaths,blue_ban1,blue_ban2,blue_ban3,blue_ban4,blue_ban5,red_ban1,red_ban2,red_ban3,red_ban4,red_ban5,Results
0,ESPORTSTMNT03/1632489,DWG KIA,oe:player:b9972f46c1e52797d66236b118d2970,Khan,Mordekaiser,1,1.000,10.000000,20435,12065,8,5,3421,oe:player:92544d7a994bd6841cccd9f6e42e3de,Canyon,Graves,5,1.0,12.000000,13944,13944,110,43,3497,oe:player:5f8ff49cd56eae61d635966b183a451,ShowMaker,Zoe,9,0.888889,14.00,16039,14274,17,24,3784,oe:player:78081d0b3c01f1f295d52d53744261a,Ghost,Miss Fortune,1,1.000000,...,5502,11465,0,0,3301,oe:player:18bdb3898e7fecb3bb0be7ff075d509,deokdam,Ezreal,5,0.8,0.4,13126,9815,5,0,3020,oe:player:b0ec3df39d61f953e7af0afd8eb8c4a,Kellin,Leona,10,0.500000,0.4,2349,5921,0,0,2137,4,24,Aatrox,Kai'Sa,Pantheon,Aphelios,Lucian,Akali,Samira,Taliyah,Gragas,Maokai,1
1,ESPORTSTMNT03/1632500,Nongshim RedForce,oe:player:6fbf57b984dbf130701e09ea4110bb8,Rich,Gragas,8,0.375,2.000000,12968,9201,0,0,3183,oe:player:dda4a873265dfd06df50a0d8a294908,Peanut,Graves,8,0.5,1.333333,7175,10362,140,2,3357,oe:player:0582fabc486c42d0db3bf52f56c6fc3,Bay,Rumble,1,0.000000,1.00,7880,10058,0,0,3338,oe:player:18bdb3898e7fecb3bb0be7ff075d509,deokdam,Kai'Sa,6,0.333333,...,24597,14009,24,0,3610,oe:player:78081d0b3c01f1f295d52d53744261a,Ghost,Miss Fortune,2,0.0,7.0,17202,13644,16,4,3161,oe:player:dee54127bb56d2e2540ba71cc416f1c,BeryL,Galio,5,0.200000,6.5,7644,7184,0,0,2004,18,7,Akali,Taliyah,Zoe,Leona,Sylas,Aatrox,Pantheon,Samira,Ekko,Orianna,0
2,ESPORTSTMNT03/1632502,Nongshim RedForce,oe:player:6fbf57b984dbf130701e09ea4110bb8,Rich,Gragas,8,0.375,2.000000,10139,8821,1,0,3110,oe:player:dda4a873265dfd06df50a0d8a294908,Peanut,Graves,8,0.5,1.250000,10554,10125,87,1,3330,oe:player:0582fabc486c42d0db3bf52f56c6fc3,Bay,Zoe,3,0.333333,0.75,8790,8353,8,0,2971,oe:player:18bdb3898e7fecb3bb0be7ff075d509,deokdam,Yasuo,1,0.000000,...,17836,13608,10,4,3463,oe:player:78081d0b3c01f1f295d52d53744261a,Ghost,Miss Fortune,2,0.0,16.0,17250,15082,21,0,3268,oe:player:dee54127bb56d2e2540ba71cc416f1c,BeryL,Galio,5,0.200000,6.5,7947,7439,0,0,2069,20,11,Akali,Taliyah,Olaf,Leona,Renekton,Aatrox,Pantheon,Samira,Kai'Sa,Ornn,0
3,6909-9183,Top Esports,oe:player:0c74b7f78409a4022a2c4c5a5ca3ee1,369,Karma,1,0.000,3.000000,18444,10711,4,0,3429,oe:player:f3322656ae1facbf4763677fe2506a3,Karsa,Pantheon,2,0.0,1.000000,8585,10106,115,3,3271,oe:player:ed29043422e3c515e1cb292464b990b,knight,Syndra,6,0.500000,2.00,13105,13284,24,10,3842,oe:player:c54e1090ef1faa736b12493051c4855,JackeyLove,Samira,3,0.333333,...,17497,12602,18,0,3324,oe:player:6d664d1402cd16be08e335feacf73b0,huanfeng,Kai'Sa,9,0.0,9.0,18459,16176,41,0,3296,oe:player:90651ebea9a35ec4e018c

In [ ]:
#Create a copy of dataframe so that changes made in new dataframe wont affect our original dataframe
df_2=df.copy()

In [ ]:
df_2.head()

,Gameid,Blue_Team_id,bluetop_playerid,bluetop_player_name,bluetop_champion,bluetop_Game_played,bluetop_champ_win_rate,bluetop_KDA,bluetop_dmg_to_cham,bluetop_goldearned,bluetop_monsterkillsownjungle,bluetop_monsterkillsenemyjungle,bluetop_goldat10,bluejng_playerid,bluejng_playername,bluejng_champion,bluejng_Game_played,bluejng_champ_win_rate,bluejng_KDA,bluejng_dmg_to_cham,bluejng_goldearned,bluejng_monsterkillsownjungle,bluejng_monsterkillsenemyjungle,bluejng_goldat10,bluemid_playerid,bluemid_playername,bluemid_champion,bluemid_Game_played,bluemid_champ_win_rate,bluemid_KDA,bluemid_dmg_to_cham,bluemid_goldearned,bluemid_monsterkillsownjungle,bluemid_monsterkillsenemyjungle,bluemid_goldat10,bluebot_playerid,bluebot_playername,bluebot_champion,bluebot_Game_played,bluebot_champ_win_rate,...,redmid_dmg_to_cham,redmid_goldearned,redmid_monsterkillsownjungle,redmid_monsterkillsenemyjungle,redmid_goldat10,redbot_playerid,redbot_playername,redbot_champion,redbot_Game_played,redbot_champ_win_rate,redbot_KDA,redbot_dmg_to_cham,redbot_goldearned,redbot_monsterkillsownjungle,redbot_monsterkillsenemyjungle,redbot_goldat10,redsup_playerid,redsup_playername,redsup_champion,redsup_Game_played,redsup_champ_win_rate,redsup_KDA,redsup_dmg_to_cham,redsup_goldearned,redsup_monsterkillsownjungle,redsup_monsterkillsenemyjungle,redsup_goldat10,Red_team_kills,Red_team_deaths,blue_ban1,blue_ban2,blue_ban3,blue_ban4,blue_ban5,red_ban1,red_ban2,red_ban3,red_ban4,red_ban5,Results
0,ESPORTSTMNT03/1632489,DWG KIA,oe:player:b9972f46c1e52797d66236b118d2970,Khan,Mordekaiser,1,1.000,10.000000,20435,12065,8,5,3421,oe:player:92544d7a994bd6841cccd9f6e42e3de,Canyon,Graves,5,1.0,12.000000,13944,13944,110,43,3497,oe:player:5f8ff49cd56eae61d635966b183a451,ShowMaker,Zoe,9,0.888889,14.00,16039,14274,17,24,3784,oe:player:78081d0b3c01f1f295d52d53744261a,Ghost,Miss Fortune,1,1.000000,...,5502,11465,0,0,3301,oe:player:18bdb3898e7fecb3bb0be7ff075d509,deokdam,Ezreal,5,0.8,0.4,13126,9815,5,0,3020,oe:player:b0ec3df39d61f953e7af0afd8eb8c4a,Kellin,Leona,10,0.500000,0.4,2349,5921,0,0,2137,4,24,Aatrox,Kai'Sa,Pantheon,Aphelios,Lucian,Akali,Samira,Taliyah,Gragas,Maokai,1
1,ESPORTSTMNT03/1632500,Nongshim RedForce,oe:player:6fbf57b984dbf130701e09ea4110bb8,Rich,Gragas,8,0.375,2.000000,12968,9201,0,0,3183,oe:player:dda4a873265dfd06df50a0d8a294908,Peanut,Graves,8,0.5,1.333333,7175,10362,140,2,3357,oe:player:0582fabc486c42d0db3bf52f56c6fc3,Bay,Rumble,1,0.000000,1.00,7880,10058,0,0,3338,oe:player:18bdb3898e7fecb3bb0be7ff075d509,deokdam,Kai'Sa,6,0.333333,...,24597,14009,24,0,3610,oe:player:78081d0b3c01f1f295d52d53744261a,Ghost,Miss Fortune,2,0.0,7.0,17202,13644,16,4,3161,oe:player:dee54127bb56d2e2540ba71cc416f1c,BeryL,Galio,5,0.200000,6.5,7644,7184,0,0,2004,18,7,Akali,Taliyah,Zoe,Leona,Sylas,Aatrox,Pantheon,Samira,Ekko,Orianna,0
2,ESPORTSTMNT03/1632502,Nongshim RedForce,oe:player:6fbf57b984dbf130701e09ea4110bb8,Rich,Gragas,8,0.375,2.000000,10139,8821,1,0,3110,oe:player:dda4a873265dfd06df50a0d8a294908,Peanut,Graves,8,0.5,1.250000,10554,10125,87,1,3330,oe:player:0582fabc486c42d0db3bf52f56c6fc3,Bay,Zoe,3,0.333333,0.75,8790,8353,8,0,2971,oe:player:18bdb3898e7fecb3bb0be7ff075d509,deokdam,Yasuo,1,0.000000,...,17836,13608,10,4,3463,oe:player:78081d0b3c01f1f295d52d53744261a,Ghost,Miss Fortune,2,0.0,16.0,17250,15082,21,0,3268,oe:player:dee54127bb56d2e2540ba71cc416f1c,BeryL,Galio,5,0.200000,6.5,7947,7439,0,0,2069,20,11,Akali,Taliyah,Olaf,Leona,Renekton,Aatrox,Pantheon,Samira,Kai'Sa,Ornn,0
3,6909-9183,Top Esports,oe:player:0c74b7f78409a4022a2c4c5a5ca3ee1,369,Karma,1,0.000,3.000000,18444,10711,4,0,3429,oe:player:f3322656ae1facbf4763677fe2506a3,Karsa,Pantheon,2,0.0,1.000000,8585,10106,115,3,3271,oe:player:ed29043422e3c515e1cb292464b990b,knight,Syndra,6,0.500000,2.00,13105,13284,24,10,3842,oe:player:c54e1090ef1faa736b12493051c4855,JackeyLove,Samira,3,0.333333,...,17497,12602,18,0,3324,oe:player:6d664d1402cd16be08e335feacf73b0,huanfeng,Kai'Sa,9,0.0,9.0,18459,16176,41,0,3296,oe:player:90651ebea9a35ec4e018c

In [ ]:
#Drop  variables having categorical values and those are only acting as an identifier from dataframe.
df_2.drop(['Gameid','Blue_Team_id','bluetop_playerid','bluetop_player_name','bluejng_playerid','bluejng_playername',\
           'bluemid_playerid','bluemid_playername','bluebot_playerid','bluebot_playername','bluesup_playerid','bluesup_playername',\
           'Red_team_id','redtop_playerid','redtop_playername','redjng_playerid','redjng_playername','redmid_playerid','redmid_playername',\
           'redbot_playerid','redbot_playername','redsup_playerid','redsup_playername'],axis=1,inplace=True)

In [ ]:
df_2.shape

(7425, 105)

In [ ]:
df_2.head()

,bluetop_champion,bluetop_Game_played,bluetop_champ_win_rate,bluetop_KDA,bluetop_dmg_to_cham,bluetop_goldearned,bluetop_monsterkillsownjungle,bluetop_monsterkillsenemyjungle,bluetop_goldat10,bluejng_champion,bluejng_Game_played,bluejng_champ_win_rate,bluejng_KDA,bluejng_dmg_to_cham,bluejng_goldearned,bluejng_monsterkillsownjungle,bluejng_monsterkillsenemyjungle,bluejng_goldat10,bluemid_champion,bluemid_Game_played,bluemid_champ_win_rate,bluemid_KDA,bluemid_dmg_to_cham,bluemid_goldearned,bluemid_monsterkillsownjungle,bluemid_monsterkillsenemyjungle,bluemid_goldat10,bluebot_champion,bluebot_Game_played,bluebot_champ_win_rate,bluebot_KDA,bluebot_dmg_to_cham,bluebot_goldearned,bluebot_monsterkillsownjungle,bluebot_monsterkillsenemyjungle,bluebot_goldat10,bluesup_champion,bluesup_Game_played,bluesup_champ_win_rate,bluesup_KDA,...,redmid_champion,redmid_Game_played,redmid_champ_win_rate,redmid_KDA,redmid_dmg_to_cham,redmid_goldearned,redmid_monsterkillsownjungle,redmid_monsterkillsenemyjungle,redmid_goldat10,redbot_champion,redbot_Game_played,redbot_champ_win_rate,redbot_KDA,redbot_dmg_to_cham,redbot_goldearned,redbot_monsterkillsownjungle,redbot_monsterkillsenemyjungle,redbot_goldat10,redsup_champion,redsup_Game_played,redsup_champ_win_rate,redsup_KDA,redsup_dmg_to_cham,redsup_goldearned,redsup_monsterkillsownjungle,redsup_monsterkillsenemyjungle,redsup_goldat10,Red_team_kills,Red_team_deaths,blue_ban1,blue_ban2,blue_ban3,blue_ban4,blue_ban5,red_ban1,red_ban2,red_ban3,red_ban4,red_ban5,Results
0,Mordekaiser,1,1.000,10.000000,20435,12065,8,5,3421,Graves,5,1.0,12.000000,13944,13944,110,43,3497,Zoe,9,0.888889,14.00,16039,14274,17,24,3784,Miss Fortune,1,1.000000,7.0,17202,13097,17,4,3540,Galio,2,0.5,16.00,...,Orianna,8,0.875,0.5,5502,11465,0,0,3301,Ezreal,5,0.8,0.4,13126,9815,5,0,3020,Leona,10,0.500000,0.4,2349,5921,0,0,2137,4,24,Aatrox,Kai'Sa,Pantheon,Aphelios,Lucian,Akali,Samira,Taliyah,Gragas,Maokai,1
1,Gragas,8,0.375,2.000000,12968,9201,0,0,3183,Graves,8,0.5,1.333333,7175,10362,140,2,3357,Rumble,1,0.000000,1.00,7880,10058,0,0,3338,Kai'Sa,6,0.333333,2.5,11921,10725,9,0,3630,Alistar,10,0.3,1.00,...,Syndra,10,0.300,14.0,24597,14009,24,0,3610,Miss Fortune,2,0.0,7.0,17202,13644,16,4,3161,Galio,5,0.200000,6.5,7644,7184,0,0,2004,18,7,Akali,Taliyah,Zoe,Leona,Sylas,Aatrox,Pantheon,Samira,Ekko,Orianna,0
2,Gragas,8,0.375,2.000000,10139,8821,1,0,3110,Graves,8,0.5,1.250000,10554,10125,87,1,3330,Zoe,3,0.333333,0.75,8790,8353,8,0,2971,Yasuo,1,0.000000,1.2,9915,11475,9,0,3063,Alistar,10,0.3,1.50,...,Yone,1,0.000,13.0,17836,13608,10,4,3463,Miss Fortune,2,0.0,16.0,17250,15082,21,0,3268,Galio,5,0.200000,6.5,7947,7439,0,0,2069,20,11,Akali,Taliyah,Olaf,Leona,Renekton,Aatrox,Pantheon,Samira,Kai'Sa,Ornn,0
3,Karma,1,0.000,3.000000,18444,10711,4,0,3429,Pantheon,2,0.0,1.000000,8585,10106,115,3,3271,Syndra,6,0.500000,2.00,13105,13284,24,10,3842,Samira,3,0.333333,1.0,20740,13742,19,0,3446,Leona,2,0.5,0.25,...,Viktor,5,0.400,8.0,17497,12602,18,0,3324,Kai'Sa,9,0.0,9.0,18459,16176,41,0,3296,Galio,3,0.333333,4.0,3865,7407,0,0,2092,9,3,Olaf,Renekton,Zoe,Twisted Fate,Orianna,Graves,Ornn,Akali,Kennen,Jayce,0
4,Jax,1,0.000,0.714286,12954,14213,19,5,3315,Hecarim,5,0.4,1.833333,14604,13537,112,1,4194,Orianna,5,0.200000,3.00,30516,16077,33,1,3367,Samira,3,0.333333,1.5,20218,16900,17,8,3767,Alistar,2,0.0,1.50,...,Viktor,5,0.400,8.5,28284,15091,45,0,3152,Kai'Sa,9,0.0,21.0,18137,18299,36,10,3353,Maokai,1,0.000000,2.5,11823,7797,0,0,2090,27,17,Olaf,Aatrox,Renekton,Galio,Leona,Graves,Ornn,Akali,Pantheon,Taliyah,0


In [ ]:
#Have a look at schemas of dataframe
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7425 entries, 0 to 7424
Columns: 105 entries, bluetop_champion to Results
dtypes: float64(20), int64(65), object(20)
memory usage: 5.9+ MB


In [ ]:
#See summary statistics of all numerical variables.
df_2.describe().T

,count,mean,std,min,25%,50%,75%,max
bluetop_Game_played,7425.0,3.089428,2.484885,1.0,1.00,2.0,4.0,17.0
bluetop_champ_win_rate,7425.0,0.533468,0.374162,0.0,0.20,0.5,1.0,1.0
bluetop_KDA,7425.0,4.763025,4.634182,0.0,1.25,3.0,7.0,27.0
bluetop_dmg_to_cham,7425.0,13729.062761,5847.416072,1323.0,9577.00,12774.0,16927.0,56514.0
bluetop_goldearned,7425.0,11693.519057,2597.568476,4795.0,9808.00,11558.0,13341.0,24050.0
...,...,...,...,...,...,...,...,...
redsup_monsterkillsenemyjungle,7425.0,0.069495,0.599944,0.0,0.00,0.0,0.0,24.0
redsup_goldat10,7425.0,2271.382492,283.948066,1757.0,2084.00,2190.0,2386.0,4307.0
Red_team_kills,7425.0,14.096027,7.701063,0.0,8.00,14.0,20.0,50.0
Red_team_deaths,7425.0,14.818451,7.399299,0.0,9.00,15.0,20.0,50.0


Champion names are in categorical format ,we need to convert it into some numeric values,therefore mapping a integar value to a champion present in whole dataframe

In [ ]:
Champion_list_total=list(df['bluetop_champion'].unique())+list(df['bluejng_champion'].unique())+list(df['bluemid_champion'].unique())+\
list(df['bluebot_champion'].unique())+list(df['bluesup_champion'].unique())+list(df['redtop_champion'].unique())+\
list(df['redjng_champion'].unique())+list(df['redmid_champion'].unique())+list(df['redbot_champion'].unique())+\
list(df['redsup_champion'].unique())+list(df['bluejng_champion'].unique())+list(df['blue_ban1'].unique())+list(df['blue_ban2'].unique())\
+list(df['blue_ban3'].unique())+list(df['blue_ban4'].unique())+list(df['blue_ban5'].unique())+list(df['red_ban1'].unique())+\
list(df['red_ban2'].unique())+list(df['red_ban3'].unique())+list(df['red_ban4'].unique())+list(df['red_ban5'].unique())



All_champion=pd.Series(Champion_list_total)
All_champion_unique=list(All_champion.unique())
len(All_champion_unique)

155

There are total 155 unique champions in dataframe from the above result.

Assign a key value pair to each champion.
Like champion name as {Key} and its mapped integar value as {value for that Key}.

In [ ]:
replace_map_comp_bluetop_champ = {'bluetop_champion' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_bluejng_champ = {'bluejng_champion' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_bluemid_champ = {'bluemid_champion' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_bluebot_champ = {'bluebot_champion' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_bluesup_champ = {'bluesup_champion' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_redtop_champ = {'redtop_champion' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_redjng_champ = {'redjng_champion' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_redmid_champ = {'redmid_champion' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_redbot_champ = {'redbot_champion' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_redsup_champ = {'redsup_champion' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_blueban1_champ = {'blue_ban1' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_blueban2_champ = {'blue_ban2' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_blueban3_champ = {'blue_ban3' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_blueban4_champ = {'blue_ban4' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_blueban5_champ = {'blue_ban5' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_redban1_champ = {'red_ban1' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_redban2_champ = {'red_ban2' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_redban3_champ = {'red_ban3' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_redban4_champ = {'red_ban4' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}
replace_map_comp_redban5_champ = {'red_ban5' : {k: v for k,v in zip(All_champion_unique,list(range(1,len(All_champion_unique)+1)))}}


#Replace champion name by integar values evaluated above.


In [ ]:
df_replaced = df_2.copy()
df_replaced.replace(replace_map_comp_bluetop_champ, inplace=True)
df_replaced.replace(replace_map_comp_bluejng_champ, inplace=True)
df_replaced.replace(replace_map_comp_bluemid_champ, inplace=True)
df_replaced.replace(replace_map_comp_bluebot_champ, inplace=True)
df_replaced.replace(replace_map_comp_bluesup_champ, inplace=True)
df_replaced.replace(replace_map_comp_redtop_champ, inplace=True)
df_replaced.replace(replace_map_comp_redjng_champ, inplace=True)
df_replaced.replace(replace_map_comp_redmid_champ, inplace=True)
df_replaced.replace(replace_map_comp_redbot_champ, inplace=True)
df_replaced.replace(replace_map_comp_redsup_champ, inplace=True)
df_replaced.replace(replace_map_comp_blueban1_champ, inplace=True)
df_replaced.replace(replace_map_comp_blueban2_champ, inplace=True)
df_replaced.replace(replace_map_comp_blueban3_champ, inplace=True)
df_replaced.replace(replace_map_comp_blueban4_champ, inplace=True)
df_replaced.replace(replace_map_comp_blueban5_champ, inplace=True)
df_replaced.replace(replace_map_comp_redban1_champ, inplace=True)
df_replaced.replace(replace_map_comp_redban2_champ, inplace=True)
df_replaced.replace(replace_map_comp_redban3_champ, inplace=True)
df_replaced.replace(replace_map_comp_redban4_champ, inplace=True)
df_replaced.replace(replace_map_comp_redban5_champ, inplace=True)

In [ ]:
df_replaced.head()

,bluetop_champion,bluetop_Game_played,bluetop_champ_win_rate,bluetop_KDA,bluetop_dmg_to_cham,bluetop_goldearned,bluetop_monsterkillsownjungle,bluetop_monsterkillsenemyjungle,bluetop_goldat10,bluejng_champion,bluejng_Game_played,bluejng_champ_win_rate,bluejng_KDA,bluejng_dmg_to_cham,bluejng_goldearned,bluejng_monsterkillsownjungle,bluejng_monsterkillsenemyjungle,bluejng_goldat10,bluemid_champion,bluemid_Game_played,bluemid_champ_win_rate,bluemid_KDA,bluemid_dmg_to_cham,bluemid_goldearned,bluemid_monsterkillsownjungle,bluemid_monsterkillsenemyjungle,bluemid_goldat10,bluebot_champion,bluebot_Game_played,bluebot_champ_win_rate,bluebot_KDA,bluebot_dmg_to_cham,bluebot_goldearned,bluebot_monsterkillsownjungle,bluebot_monsterkillsenemyjungle,bluebot_goldat10,bluesup_champion,bluesup_Game_played,bluesup_champ_win_rate,bluesup_KDA,...,redmid_champion,redmid_Game_played,redmid_champ_win_rate,redmid_KDA,redmid_dmg_to_cham,redmid_goldearned,redmid_monsterkillsownjungle,redmid_monsterkillsenemyjungle,redmid_goldat10,redbot_champion,redbot_Game_played,redbot_champ_win_rate,redbot_KDA,redbot_dmg_to_cham,redbot_goldearned,redbot_monsterkillsownjungle,redbot_monsterkillsenemyjungle,redbot_goldat10,redsup_champion,redsup_Game_played,redsup_champ_win_rate,redsup_KDA,redsup_dmg_to_cham,redsup_goldearned,redsup_monsterkillsownjungle,redsup_monsterkillsenemyjungle,redsup_goldat10,Red_team_kills,Red_team_deaths,blue_ban1,blue_ban2,blue_ban3,blue_ban4,blue_ban5,red_ban1,red_ban2,red_ban3,red_ban4,red_ban5,Results
0,1,1,1.000,10.000000,20435,12065,8,5,3421,46,5,1.0,12.000000,13944,13944,110,43,3497,96,9,0.888889,14.00,16039,14274,17,24,3784,125,1,1.000000,7.0,17202,13097,17,4,3540,51,2,0.5,16.00,...,34,8,0.875,0.5,5502,11465,0,0,3301,75,5,0.8,0.4,13126,9815,5,0,3020,138,10,0.500000,0.4,2349,5921,0,0,2137,4,24,9,126,22,129,43,11,127,81,2,31,1
1,2,8,0.375,2.000000,12968,9201,0,0,3183,46,8,0.5,1.333333,7175,10362,140,2,3357,29,1,0.000000,1.00,7880,10058,0,0,3338,126,6,0.333333,2.5,11921,10725,9,0,3630,137,10,0.3,1.00,...,97,10,0.300,14.0,24597,14009,24,0,3610,125,2,0.0,7.0,17202,13644,16,4,3161,51,5,0.200000,6.5,7644,7184,0,0,2004,18,7,11,81,96,138,42,9,22,127,83,34,0
2,2,8,0.375,2.000000,10139,8821,1,0,3110,46,8,0.5,1.250000,10554,10125,87,1,3330,96,3,0.333333,0.75,8790,8353,8,0,2971,55,1,0.000000,1.2,9915,11475,9,0,3063,137,10,0.3,1.50,...,14,1,0.000,13.0,17836,13608,10,4,3463,125,2,0.0,16.0,17250,15082,21,0,3268,51,5,0.200000,6.5,7947,7439,0,0,2069,20,11,11,81,44,138,16,9,22,127,126,8,0
3,3,1,0.000,3.000000,18444,10711,4,0,3429,22,2,0.0,1.000000,8585,10106,115,3,3271,97,6,0.500000,2.00,13105,13284,24,10,3842,127,3,0.333333,1.0,20740,13742,19,0,3446,138,2,0.5,0.25,...,68,5,0.400,8.0,17497,12602,18,0,3324,126,9,0.0,9.0,18459,16176,41,0,3296,51,3,0.333333,4.0,3865,7407,0,0,2092,9,3,44,16,96,98,34,46,8,11,7,33,0
4,4,1,0.000,0.714286,12954,14213,19,5,3315,50,5,0.4,1.833333,14604,13537,112,1,4194,34,5,0.200000,3.00,30516,16077,33,1,3367,127,3,0.333333,1.5,20218,16900,17,8,3767,137,2,0.0,1.50,...,68,5,0.400,8.5,28284,15091,45,0,3152,126,9,0.0,21.0,18137,18299,36,10,3353,31,1,0.000000,2.5,11823,7797,0,0,2090,27,17,44,9,16,51,138,46,8,11,22,81,0


In [ ]:
df_replaced.shape

(7425, 105)

In [ ]:
a=[(i,j) for i,j in enumerate(df_replaced.columns)]
a

[(0, 'bluetop_champion'),
 (1, 'bluetop_Game_played'),
 (2, 'bluetop_champ_win_rate'),
 (3, 'bluetop_KDA'),
 (4, 'bluetop_dmg_to_cham'),
 (5, 'bluetop_goldearned'),
 (6, 'bluetop_monsterkillsownjungle'),
 (7, 'bluetop_monsterkillsenemyjungle'),
 (8, 'bluetop_goldat10'),
 (9, 'bluejng_champion'),
 (10, 'bluejng_Game_played'),
 (11, 'bluejng_champ_win_rate'),
 (12, 'bluejng_KDA'),
 (13, 'bluejng_dmg_to_cham'),
 (14, 'bluejng_goldearned'),
 (15, 'bluejng_monsterkillsownjungle'),
 (16, 'bluejng_monsterkillsenemyjungle'),
 (17, 'bluejng_goldat10'),
 (18, 'bluemid_champion'),
 (19, 'bluemid_Game_played'),
 (20, 'bluemid_champ_win_rate'),
 (21, 'bluemid_KDA'),
 (22, 'bluemid_dmg_to_cham'),
 (23, 'bluemid_goldearned'),
 (24, 'bluemid_monsterkillsownjungle'),
 (25, 'bluemid_monsterkillsenemyjungle'),
 (26, 'bluemid_goldat10'),
 (27, 'bluebot_champion'),
 (28, 'bluebot_Game_played'),
 (29, 'bluebot_champ_win_rate'),
 (30, 'bluebot_KDA'),
 (31, 'bluebot_dmg_to_cham'),
 (32, 'bluebot_goldearned'),

#Model-1
Hypothesis- There exist a relationship between champions picked by players at different position in both teams with Result of the match(either win or loss)

**Variables taken**:\
bluetop_champion,bluejng_champion,bluemid_champion,bluebot_champion,bluesup_champion,redtop_champion,redjng_champion,redmid_champion,redbot_champion,redsup_champion     

In [ ]:
data_X=df_replaced[df_replaced.columns[[0,9,18,27,36,47,56,65,74,83]] ] 
data_X.shape

(7425, 10)

In [ ]:
data_y=df_replaced['Results']


In [ ]:
#Using stats Logit model
X_train,X_test,y_train,y_test=train_test_split(data_X,data_y,random_state=123,test_size=0.3)
X_train1=sm.add_constant(X_train)
clf=sm.Logit(y_train,X_train1).fit()
print(clf.summary())

Optimization terminated successfully.
         Current function value: 0.688108
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                Results   No. Observations:                 5197
Model:                          Logit   Df Residuals:                     5186
Method:                           MLE   Df Model:                           10
Date:                Sat, 26 Feb 2022   Pseudo R-squ.:                0.004284
Time:                        14:39:04   Log-Likelihood:                -3576.1
converged:                       True   LL-Null:                       -3591.5
Covariance Type:            nonrobust   LLR p-value:                 0.0006407
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.3279      0.251      1.305      0.192      -0.165       0.820
bluetop_cha

From the above output of Logistic regression,we can see that as all the p values of attributes are greater than 0.05 if we are considering significance level of 5 % except "redjng_champion" "red_mid_champion" and "red_sup_champion",therefore it means most of the above taken variables are insignificant.
So our hypothesis came out to be wrong,there is very weak  relation between champion picked at different position with game result.

In [ ]:
print(clf.bic)
print(clf.aic)

7246.308454758487
7174.194249793394


#Model-2

Hypothesis-There is a relationship between champions banned by blue and red teams with result of the game.

**Variables Taken:**\
blue_ban1,blue_ban2,blue_ban3 ,blue_ban4 ,blue_ban5 ,red_ban1 ,red_ban2 ,red_ban3 ,red_ban4 ,red_ban5       

In [ ]:
data_X=df_replaced[df_replaced.columns[[94,95,96,97,98,99,100,101,102,103]] ] #[6,54x,]
data_X.shape

(7425, 10)

In [ ]:
data_y=df_replaced['Results']


In [ ]:

X_train,X_test,y_train,y_test=train_test_split(data_X,data_y,random_state=123,test_size=0.3)
X_train1=sm.add_constant(X_train)
clf=sm.Logit(y_train,X_train1).fit()
print(clf.summary())

Optimization terminated successfully.
         Current function value: 0.689923
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                Results   No. Observations:                 5197
Model:                          Logit   Df Residuals:                     5186
Method:                           MLE   Df Model:                           10
Date:                Fri, 25 Feb 2022   Pseudo R-squ.:                0.001657
Time:                        17:29:08   Log-Likelihood:                -3585.5
converged:                       True   LL-Null:                       -3591.5
Covariance Type:            nonrobust   LLR p-value:                    0.2916
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2539      0.133      1.914      0.056      -0.006       0.514
blue_ban1      0.0004      0.

Above result is reflecting that ,all the variables are none significant if we take 95 % confidence interval except "red_ban2",but if we increase confidence interval to 99% that too will become insignificant.
So there is no relationship between champions banned by both teams with  the result of the game.

#Model-3

**Hypothesis**-Is there any relationship between variables "bluetop_Game_played","bluetop_goldearned","bluetop_monsterkillsenemyjungle","bluetop_goldat10","bluejng_KDA","Blue_team_kills","redtop_Game_played","redtop_dmg_to_cham","Red_team_kills" with result of the game.

In [ ]:
data_X=df_replaced[df_replaced.columns[[1,5,7,8,12,45,48,51,92]] ] 
data_X.shape

(7425, 9)

#Use standardscalar to scale the variables as it doesn't contain any variable that we map values above.So it wont affect our mapped integar values

In [ ]:
X_scaled=StandardScaler().fit_transform(data_X)
X_scaled=pd.DataFrame(X_scaled,columns=data_X.columns)


In [ ]:
data_y=df_replaced['Results']


In [ ]:

X_train,X_test,y_train,y_test=train_test_split(X_scaled,data_y,random_state=123,test_size=0.3)
X_train1=sm.add_constant(X_train)
clf=sm.Logit(y_train,X_train1).fit()
print(clf.summary())

Optimization terminated successfully.
         Current function value: 0.092482
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                Results   No. Observations:                 5197
Model:                          Logit   Df Residuals:                     5187
Method:                           MLE   Df Model:                            9
Date:                Sun, 27 Feb 2022   Pseudo R-squ.:                  0.8662
Time:                        08:18:19   Log-Likelihood:                -480.63
converged:                       True   LL-Null:                       -3591.5
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               0.5562      0.109      5.087 

In [ ]:
print(clf.bic)
print(clf.aic)

1046.8127614796635
981.2543933295792


#Interpretation from the model result:
As we can see all variables are significant because all having p values less than 0.05 ,so they are having a relationship with Result of the game.\
Now look the coefficients of "Blue_team_kills" 4.0426 which is very high,it means for every one unit increase in value of"Blue_team_kills",the odds of blue team wins multiplies by exp(4.0426) that is 56 times.\

Red team kills is exactly same but inverse of Blue team kills,with the increase of red team kills,the odds of Blue team winning would reduce.\
Gold earned by bluetop player is also significant,means more gold earned by bluetop player higher is odds of winning the game for blueteam.

Value of psuedo R square is good near to one.

In [ ]:
#Create instance for LogisticRegression model
lr=LogisticRegression(max_iter=500)

#Fit model on training data
lr=lr.fit(X_train,y_train)

In [ ]:
#Predict the result for Test data
y_pred=lr.predict(X_test)

In [ ]:
#Get Accuracy for test data 
lr.score(X_test,y_test)

0.9542190305206463

In [ ]:
# Constructing the confusion matrix.
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 982,   51],
       [  51, 1144]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95      1033
           1       0.96      0.96      0.96      1195

    accuracy                           0.95      2228
   macro avg       0.95      0.95      0.95      2228
weighted avg       0.95      0.95      0.95      2228



#Using Decision Tree Algorithm:
 

In [ ]:
data_X=df_replaced[df_replaced.columns[[1,5,7,8,12,45,48,51,92]] ] 
data_X.shape

(7425, 9)

In [ ]:
data_y=df_replaced['Results']


In [ ]:
X_scaled=StandardScaler().fit_transform(data_X)
X_scaled=pd.DataFrame(X_scaled,columns=data_X.columns)


In [ ]:

X_train,X_test,y_train,y_test=train_test_split(data_X,data_y,random_state=123,test_size=0.3)


In [ ]:
#Create instance for Decision Tree classifier model
dec_clf=DecisionTreeClassifier()

#Fit model on training data
dec_clf=dec_clf.fit(X_train,y_train)

In [ ]:
#Predict the result for Test data
y_pred=m.predict(X_test)

In [ ]:
#Get Accuracy for test data 
dec_clf.score(X_test,y_test)

0.9429982046678635

In [ ]:
# Constructing the confusion matrix.
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 970,   63],
       [  67, 1128]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1033
           1       0.95      0.94      0.95      1195

    accuracy                           0.94      2228
   macro avg       0.94      0.94      0.94      2228
weighted avg       0.94      0.94      0.94      2228

